
# What Drives Used-Car Prices? — Linear Models + SFS=6 (Log Target, Log-TEs, **Train-Mode Categorical Imputation**)

**Date:** 2025-08-27

This notebook follows your rubric and **does not** use a global `"unknown"` category.  
Categorical NaNs are imputed with the **mode computed on the TRAIN split** to avoid leakage.

**Pipeline**
1) **EDA**: shape, head, `info()`, missing %, zeros %, numeric distributions, heat map  
2) **Prep**: drop redundant columns, columns-first missingness (on raw), row hygiene, outlier rules (no MPY), imputations  
3) **Encoding**: OOF James–Stein on **log(price)** for `manufacturer`, `type`, `fuel`, `state`  
4) **Feature selection**: SFS = 6 from numeric/ordinal + TE features  
5) **Models**: OLS, Ridge, Lasso, ElasticNet, Huber on log(target) + Duan smearing → dollars  
6) **Diagnostics**: summary CSV + top absolute errors CSV


In [ ]:

import warnings, os, re, math
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Paths & constants
DATA_DIR = Path('data'); IMG_DIR = Path('images')
DATA_DIR.mkdir(exist_ok=True); IMG_DIR.mkdir(exist_ok=True)

RANDOM_STATE=42; CV_SPLITS=5; CURRENT_YEAR=2025
PRICE_MIN, PRICE_MAX = 3_000, 100_000
AGE_MAX, ODO_MAX = 30, 300_000

OUT_SUMMARY = DATA_DIR/'model_summary.csv'
OUT_ERRORS  = DATA_DIR/'top_abs_errors.csv'
OUT_CLEAN   = DATA_DIR/'vehicles_clean.csv'


## 1) Data Understanding (EDA)

In [ ]:

# Load raw
path = DATA_DIR/'vehicles.csv'
assert path.exists(), "Upload your dataset to data/vehicles.csv (relative to this notebook)"
df0 = pd.read_csv(path)
print("Raw shape:", df0.shape)
display(df0.head())

print("\n.info():")
df0.info()

def missing_table(df: pd.DataFrame):
    n = len(df)
    miss_ct = df.isna().sum()
    miss_pct = (miss_ct / max(n,1) * 100).round(2)
    return pd.DataFrame({'missing_count': miss_ct, 'missing_pct': miss_pct}).sort_values('missing_pct', ascending=False)

def zeros_table(df: pd.DataFrame):
    num_cols = df.select_dtypes(include=[np.number]).columns
    z = {}
    n = len(df)
    for c in num_cols:
        z[c] = int((df[c]==0).sum())
    out = pd.DataFrame({'zeros_count': pd.Series(z)})
    out['zeros_pct'] = (out['zeros_count']/max(n,1)*100).round(2)
    return out.sort_values('zeros_pct', ascending=False)

print("\nMissingness (raw):"); display(missing_table(df0).head(30))
print("\nZero-filled numerics (raw):"); display(zeros_table(df0).head(30))

# Numeric distributions & correlation heat map
num_cols = df0.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c not in ['lat','long']]

if len(num_cols) >= 1:
    n = min(6, len(num_cols))
    fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(8, 3*n))
    if n == 1: axes=[axes]
    for ax, c in zip(axes, num_cols[:n]):
        sns.histplot(df0[c].dropna(), bins=40, kde=False, ax=ax)
        ax.set_title(f"Distribution: {c}")
    fig.tight_layout()
    plt.savefig(IMG_DIR/'numeric_distributions_top6.png', dpi=120)
    plt.close(fig)
else:
    print("No numeric columns found for histograms.")

if len(num_cols) >= 2:
    corr = df0[num_cols].corr()
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, cmap='vlag', center=0, annot=False)
    plt.title("Correlation Heat Map (numeric)")
    plt.tight_layout()
    plt.savefig(IMG_DIR/'corr_heatmap.png', dpi=120)
    plt.close()
else:
    print("Fewer than 2 numeric columns; skipping correlation heat map.")


Raw shape: (267873, 18)


,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc



.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267873 entries, 0 to 267872
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            267873 non-null  int64  
 1   region        267873 non-null  object 
 2   price         267873 non-null  int64  
 3   year          266866 non-null  float64
 4   manufacturer  256393 non-null  object 
 5   model         264604 non-null  object 
 6   condition     160089 non-null  object 
 7   cylinders     157321 non-null  object 
 8   fuel          266105 non-null  object 
 9   odometer      265079 non-null  float64
 10  title_status  262438 non-null  object 
 11  transmission  266275 non-null  object 
 12  VIN           165339 non-null  object 
 13  drive         183710 non-null  object 
 14  size          76228 non-null   object 
 15  type          207809 non-null  object 
 16  paint_color   186977 non-null  object 
 17  state         267872 non-null  object 

,missing_count,missing_pct
size,191645,71.54
cylinders,110552,41.27
condition,107784,40.24
VIN,102534,38.28
drive,84163,31.42
paint_color,80896,30.20
type,60064,22.42
manufacturer,11480,4.29
title_status,5435,2.03
model,3269,1.22



Zero-filled numerics (raw):


,zeros_count,zeros_pct
price,18943,7.07
odometer,1352,0.50
id,0,0.00
year,0,0.00


## 2) Data Preparation

In [ ]:

df = df0.copy()

# 2.1 Drop redundant columns and 'model' (too messy)
drop_cols_fixed = [c for c in ['id','ID','vin','VIN','region','Region','model'] if c in df.columns]
df.drop(columns=drop_cols_fixed, inplace=True, errors='ignore')

# 2.2 Columns-first missing drop (decide on RAW); keep ordinals even if sparse
protect = set(['condition','cylinders','title_status','drive'])
miss0 = df0.isna().mean()
high_miss_cols0 = [c for c,p in miss0.items() if p>0.70 and c not in protect and c in df.columns]
df.drop(columns=high_miss_cols0, inplace=True, errors='ignore')

# 2.3 Coerce numerics
for c in ['price','year','odometer']:
    if c in df.columns: df[c] = pd.to_numeric(df[c], errors='coerce')

# 2.4 Row hygiene
bad = (df['price'].isna()| (df['price']==0)) & (df['year'].isna()| (df['year']==0))
df = df.loc[~bad].copy()
if 'odometer' in df.columns:
    bad2 = (df['year'].isna()| (df['year']==0)) & (df['odometer'].isna()| (df['odometer']==0))
    df = df.loc[~bad2].copy()

# 2.5 Outlier rules (no MPY)
df = df[(df['price']>=PRICE_MIN) & (df['price']<=PRICE_MAX)]
df['age'] = CURRENT_YEAR - df['year']
df = df[(df['age'].isna()) | (df['age']<=AGE_MAX)]
df = df[(df['odometer'].isna()) | (df['odometer']<=ODO_MAX)]
df.loc[df['odometer']==0, 'odometer']=np.nan

# 2.6 Normalize text
def norm(s): return s.astype(str).str.strip().str.lower()
for c in ['manufacturer','fuel','type','state','condition','cylinders','title_status','drive']:
    if c in df.columns: df[c]=norm(df[c])

# 2.7 Ordinals
cond_map  = {'salvage':0,'fair':1,'good':2,'excellent':3,'like new':4,'new':5}
title_map = {'salvage':0,'rebuilt':1,'lien':2,'missing':2,'parts only':2,'clean':3}
drive_map = {'fwd':0,'rwd':1,'4wd':2,'awd':2}

#the parse_cyl(s) return the first/first+ digit in the string.
#re is the standard library of and let us to do search.
#this function is mainly for cylinders column
def parse_cyl(x):
    if pd.isna(x): return np.nan
    m = re.search(r'(\d{1,2})', str(x))
    return float(m.group(1)) if m else np.nan

df['condition_ord']    = df['condition'].map(cond_map)    if 'condition'    in df.columns else np.nan
df['title_status_ord'] = df['title_status'].map(title_map)if 'title_status' in df.columns else np.nan
df['drive_ord']        = df['drive'].map(drive_map)       if 'drive'        in df.columns else np.nan
df['cylinders_num']    = df['cylinders'].map(parse_cyl)   if 'cylinders'    in df.columns else np.nan

# 2.8 Impute numerics (median)
for c in ['year','odometer','age','condition_ord','title_status_ord','drive_ord','cylinders_num']:
    if c in df.columns:
        med = float(np.nanmedian(df[c])) if df[c].notna().any() else 0.0   #median ignore nan
        df[c] = df[c].fillna(med)

# NOTE: Do NOT fill categoricals here (we'll impute with TRAIN MODE later).

# 2.9 Engineered features
df['odometer_10k']    = df['odometer']/10_000.0
df['odometer_10k_sq'] = (df['odometer_10k']**2).astype(float)

# Optional year bins
if 'year' in df.columns:
    df['year_bin'] = pd.cut(df['year'], bins=[1980,2000,2005,2010,2015,2020,2025],
                            labels=False, include_lowest=True).fillna(-1).astype(int)

df.to_csv(OUT_CLEAN, index=False)
print("Cleaned shape:", df.shape, "| saved:", OUT_CLEAN)


Cleaned shape: (220341, 21) | saved: data/vehicles_clean.csv


## 3) OOF James–Stein encoders on **log(price)** (TRAIN-mode fills for categoricals)

In [ ]:

from sklearn.model_selection import KFold

def oof_te_fit_transform_log(x_train, y_train_log, alpha=8.0, n_splits=5, random_state=42):
    s = x_train.astype(str).fillna('<NA>')
    idx = np.arange(len(s))
    #instantiate a KFold object
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    #create a y log space
    y = y_train_log.astype(float)
    mu_g = float(np.mean(y))
    oof = pd.Series(index=s.index, dtype=float)
    #tr:fold's training row, va:fold's validation row
    for tr, va in kf.split(idx):
        s_tr, y_tr = s.iloc[tr], y[tr]
        mu = float(np.mean(y_tr))
        g = pd.DataFrame({'cat':s_tr, 'y':y_tr}).groupby('cat')['y'].agg(['mean','size'])

        #encoding based on James-Stein algorithm
        enc = ((g['size']*g['mean'] + alpha*mu) / (g['size']+alpha)).to_dict()  #put to dictionary, used as look up table
        #fill validification row of oof with mean value
        oof.iloc[va] = s.iloc[va].map(enc).fillna(mu).values

    g_full = pd.DataFrame({'cat':s, 'y':y}).groupby('cat')['y'].agg(['mean','size'])  #final data set, not cv data set
    enc_full = ((g_full['size']*g_full['mean'] + alpha*mu_g) / (g_full['size']+alpha)).to_dict()  # calculate enc value for final data set
    oof = oof.fillna(s.map(enc_full)).fillna(mu_g)
    return oof.astype(float), (enc_full, mu_g)  # return oof, full enc and mean

def te_transform_from_map(x_series, mapping_and_mu):
    enc_full, mu = mapping_and_mu
    s = x_series.astype(str).fillna('<NA>')
    return s.map(enc_full).fillna(mu).astype(float)


## 4) Train/Test Split & SFS=6 Pool

In [ ]:

# Split
tr_idx, te_idx = train_test_split(df.index, test_size=0.2, random_state=RANDOM_STATE)
y_tr = df.loc[tr_idx, 'price'].values.astype(float)
y_te = df.loc[te_idx, 'price'].values.astype(float)
ytr_log = np.log(y_tr.clip(PRICE_MIN, PRICE_MAX))

# TRAIN-mode fills for categoricals (no leakage)
cat_cols = [c for c in ['manufacturer','fuel','type','state'] if c in df.columns]
mode_fills = {}
for c in cat_cols:
    m = df.loc[tr_idx, c].dropna().mode()
    mode_fills[c] = str(m.iloc[0]) if len(m) else 'unknown'

# Base numeric/ordinal candidates
base_cols = ['year','year_bin','age','odometer_10k','odometer_10k_sq',
             'condition_ord','cylinders_num','title_status_ord','drive_ord']
base_cols = [c for c in base_cols if c in df.columns]

Xtr = df.loc[tr_idx, base_cols].copy()
Xte = df.loc[te_idx, base_cols].copy()

# TEs on simple categories with TRAIN-mode fill
encs = {}
for col, alpha in [('manufacturer',8.0), ('fuel',10.0), ('type',12.0), ('state',15.0)]:
    if col in df.columns:
        #fill NaN with mode
        xtr = df.loc[tr_idx, col].fillna(mode_fills[col])
        #calculate JS value
        oof_vals, mp = oof_te_fit_transform_log(xtr, ytr_log, alpha=alpha, n_splits=CV_SPLITS, random_state=RANDOM_STATE)
        #assign the JS value column
        encs[f'js_{col}'] = (oof_vals, mp)

for name,(oof_vals, mp) in encs.items():
    Xtr[name] = oof_vals.values
    src = name.replace('js_','')
    Xte[name] = te_transform_from_map(df.loc[te_idx, src].fillna(mode_fills[src]), mp).values

print("SFS candidates:", len(Xtr.columns))
print(sorted(list(Xtr.columns)))


SFS candidates: 13
['age', 'condition_ord', 'cylinders_num', 'drive_ord', 'js_fuel', 'js_manufacturer', 'js_state', 'js_type', 'odometer_10k', 'odometer_10k_sq', 'title_status_ord', 'year', 'year_bin']


In [ ]:

from sklearn.linear_model import Ridge

candidates = list(Xtr.columns)
selected = []

def cv_score(cols):
    pipe = Pipeline([('imp', SimpleImputer(strategy='median')),
                     ('sc', StandardScaler()),
                     ('reg', Ridge(alpha=1.0))])
    scores = cross_val_score(pipe, Xtr[cols], ytr_log,
                             cv=KFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE),
                             scoring='neg_mean_squared_error')
    return float(np.mean(scores))

target_k = min(6, len(candidates))
while len(selected) < target_k and candidates:
    best_c, best_sc = None, -np.inf
    for c in candidates:
        sc = cv_score(selected + [c])
        if sc > best_sc:
            best_sc, best_c = sc, c
    selected.append(best_c); candidates.remove(best_c)
    print(f"Add: +{best_c} (CV neg-MSE={best_sc:.6f})")

print("Selected 6:", selected)
if len(selected) < 6:
    print(f"WARNING: Only {len(selected)} features available for SFS: {len(selected)}")


Add: +year (CV neg-MSE=-0.298662)
Add: +cylinders_num (CV neg-MSE=-0.234484)
Add: +odometer_10k (CV neg-MSE=-0.205274)
Add: +drive_ord (CV neg-MSE=-0.180440)
Add: +js_fuel (CV neg-MSE=-0.160544)
Add: +js_manufacturer (CV neg-MSE=-0.151261)
Selected 6: ['year', 'cylinders_num', 'odometer_10k', 'drive_ord', 'js_fuel', 'js_manufacturer']


## 5) Models (log target) → dollars via Duan smearing

In [ ]:

models = [
    ('OLS', LinearRegression(), None),
    ('Ridge', Ridge(), {'reg__alpha':[1e-3,1e-2,1e-1,1,10,100]}),
    ('Lasso', Lasso(max_iter=20000), {'reg__alpha':[1e-4,1e-3,1e-2,1e-1,1]}),
    ('ElasticNet', ElasticNet(max_iter=20000), {'reg__alpha':[1e-4,1e-3,1e-2,1e-1], 'reg__l1_ratio':[0.1,0.5,0.9]}),
    ('Huber', HuberRegressor(), {'reg__epsilon':[1.2,1.5,1.8,2.5], 'reg__alpha':[1e-6,1e-5,1e-4,1e-3]}),
]

# calculate and return the smear value, where smear*np.exp(pred_log)=pred_price
def duan_smear(y_true_log, y_pred_log):
    resid = y_true_log - y_pred_log
    return float(np.mean(np.exp(resid)))

# Trim the top 2% and trim bottom 2% based on the absolute error
# and return the rmse and mae of the rest
def trimmed(y_true, y_pred, p=0.02):
    ae = np.abs(y_true - y_pred)
    n=len(ae)
    k=int(np.floor(p*n)) #number need to be trimmed
    if k>0 and 2*k<n:
        keep = np.argsort(ae)[k:n-k]  #sort the list and keep from k to n-k
        yt, yp = y_true[keep], y_pred[keep]
    else:
        yt, yp = y_true, y_pred

    #calculate the rmse and mae
    rmse = float(np.sqrt(mean_squared_error(yt, yp)))
    mae  = float(mean_absolute_error(yt, yp))
    return rmse, mae

rows, preds = [], {}
for name, est, grid in models:
    pipe = Pipeline([('imp', SimpleImputer(strategy='median')),
                     ('sc', StandardScaler()),
                     ('reg', est)])  #estimator could be OLS/Ridge/Lasso/ElasticNet/Huber
    if grid is not None:
        gs = GridSearchCV(pipe, grid, scoring='neg_mean_squared_error',
                          cv=KFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE),
                          refit=True)
        gs.fit(Xtr[selected], ytr_log)
        est_fit = gs.best_estimator_; params = gs.best_params_
    else:
        pipe.fit(Xtr[selected], ytr_log); est_fit = pipe; params = {}

    ytr_pred_log = est_fit.predict(Xtr[selected])
    smear = duan_smear(ytr_log, ytr_pred_log)

    yte_pred_log = est_fit.predict(Xte[selected])

    #np.exp(yte_pred_log) * smear = pred_price in dollar value
    p_te = np.clip(np.exp(yte_pred_log) * smear, PRICE_MIN, PRICE_MAX)

    rmse = float(np.sqrt(mean_squared_error(y_te, p_te)))
    mae  = float(mean_absolute_error(y_te, p_te))
    r2d  = float(r2_score(y_te, p_te))
    r2l  = float(r2_score(np.log(y_te.clip(PRICE_MIN,PRICE_MAX)), yte_pred_log))
    rmse_t, mae_t = trimmed(y_te, p_te, p=0.02)

    rows.append({'Model':name, 'Best Params':params, 'Selected 6':', '.join(selected),
                 'RMSE($)':rmse, 'MAE($)':mae, 'R²_log':r2l, 'R²($)':r2d,
                 'RMSE_trim2%($)':rmse_t, 'MAE_trim2%($)':mae_t})
    preds[name] = p_te

summary = pd.DataFrame(rows).sort_values('RMSE($)').reset_index(drop=True)
display(summary)
summary.to_csv(OUT_SUMMARY, index=False); print("Saved:", OUT_SUMMARY)

best_name = summary.iloc[0]['Model']
best_pred = preds[best_name]


,Model,Best Params,Selected 6,RMSE($),MAE($),R²_log,R²($),RMSE_trim2%($),MAE_trim2%($)
0,Lasso,{'reg__alpha': 0.0001},"year, cylinders_num, odometer_10k, drive_ord, ...",8226.427527,5570.226092,0.719299,0.651482,6913.863070,5142.256189
1,ElasticNet,"{'reg__alpha': 0.0001, 'reg__l1_ratio': 0.1}","year, cylinders_num, odometer_10k, drive_ord, ...",8227.170383,5570.619506,0.719297,0.651420,6914.561648,5142.605470
2,Ridge,{'reg__alpha': 10},"year, cylinders_num, odometer_10k, drive_ord, ...",8227.328854,5570.704568,0.719297,0.651406,6914.716973,5142.682827
3,OLS,{},"year, cylinders_num, odometer_10k, drive_ord, ...",8227.432476,5570.760447,0.719297,0.651397,6914.822807,5142.734963
4,Huber,"{'reg__alpha': 0.001, 'reg__epsilon': 2.5}","year, cylinders_num, odometer_10k, drive_ord, ...",8239.091421,5577.738274,0.719164,0.650409,6929.179286,5149.715760


Saved: data/model_summary.csv


## 6) Diagnostics — Worst absolute errors

In [ ]:

te = df.loc[te_idx].copy()
desired = ['price','year','odometer','odometer_10k','odometer_10k_sq','year_bin','age',
           'cylinders','cylinders_num','condition','condition_ord','title_status','title_status_ord',
           'drive','drive_ord','manufacturer','fuel','type','state']
cols = [c for c in desired if c in te.columns]

report = te[cols].copy().rename(columns={'price':'true_price'})
report['pred_price'] = best_pred
report['abs_err'] = np.abs(report['true_price'] - report['pred_price'])

top = report.sort_values('abs_err', ascending=False).head(25)
display(top)
top.to_csv(OUT_ERRORS, index=False); print("Saved:", OUT_ERRORS)


## 7) Findings & Next Steps
- See README for non-technical summary.